In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np

#DEBUG = True
DEBUG = False

#SHOW = True
SHOW = False

SAVE_IMG = True
#SAVE_IMG = False

# OpenCV: BGR; Matplot: RGB
def show_image(img):
    if (SHOW == True):
        RGB_im = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(RGB_im)
        plt.show()

In [ ]:
def extract_img(img):
    new_img = img[:, :, 1] # The G part is clearer
    #show_image(new_img)
    return new_img

## Create the Image dir
The directory is used to put cutted image

In [ ]:
def create_dir(parent_dir, directory):
    path = os.path.join(parent_dir, directory) 
    
    if (os.path.isdir(path) == True):
        return None

    try:
        os.mkdir(path)
        return path
    except OSError as error: 
        print(error)
        return None

## Save the Image
Called by split_img()

In [ ]:
def save_img(img, fn):
    if (SAVE_IMG == False):
        return
    print("save_img: ", fn)
    name = fn + ".jpg"
    cv2.imwrite(name, img)
    return name

## Split the image into 4 digit image
### Parameters
- img: return from cv2.imread
- path: The directory for putting cutted image
- filename: The image file name

### Return
An array contains saved cutted image name

In [ ]:
def split_img(img, dir_path, filename):
    saved_file_name = []

    if img is None:
        return

    rows = img.shape[1]

    step = rows//4
    # if step is too small which means the picture is too small, skip it
    if (step < 100):
        return
    
    name = os.path.splitext(filename)
    
    old_row = 0
    for i in range(1, 5):
        small_img = img[:, old_row:step*i]
        #show_image(small_img)
        old_row = step*i
        
        # save cutted image
        img_name = name[0] + "-" + str(i)
        full_path = os.path.join(dir_path, img_name)
        small_img_name = save_img(small_img, full_path)
        # store the cutted image file name
        saved_file_name.append(small_img_name)

    return saved_file_name

## Split the image by contours

### Store each contours (cutted image)

In [ ]:
def store_contours(img, arr, path, filename):
    fn = None

    for idx, (x,y,w,h) in enumerate(arr):
        roi = img[y:y+h, x:x+w]
        thresh = roi.copy()
        print("store_contours: show each contours")
        show_image(thresh)

        main_fn =  os.path.splitext(filename)
        fn = os.path.join(path, '{}-{}'.format(main_fn[0], idx+1))
        fn = save_img(thresh, fn)

    return fn

### Show the contours on original image

In [ ]:
def show_contours(img, contours):
    contours_img = img.copy()
    cv2.drawContours(contours_img, contours, -1, (255, 0, 0), 3)
    show_image(contours_img)

In [ ]:
def split_img_contours(img, dir_path, filename):

    if img is None:
        return

    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    otsu, thresh = cv2.threshold(imgray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # RETR_EXTERNAL: only outer flag
    # otherwise, 0 will have two contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    show_contours(img, contours)

    cnts = sorted([(c, cv2.boundingRect(c)[0]) for c in contours], key=lambda x:x[1])
    if (DEBUG == True):
        print("Found {} contours".format(len(cnts)))

    # Store the contours
    con_arr = []
    for (c, _) in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        if (DEBUG == True):
            print("contours: ", (x, y, w, h))
        
        if (h >= 100 and w >= 30):
            con_arr.append((x,y,w,h))
    
    return store_contours(img, con_arr, dir_path, filename)

In [ ]:
DIR = 'test_images/'

for root, subdirs, files in os.walk(DIR):
    print("root:", root)
    for filename in files:
        file_path = os.path.join(root, filename)
        if file_path.endswith(".jpg"):
            print("Process: ", file_path)
            
            # FIXME: create dir moves to store image
            # filename = "dir name" + ".jpg"
            dir_name = os.path.splitext(filename)
            path = create_dir(DIR, dir_name[0])
            if path is None:
                continue
            
            img = cv2.imread(file_path, cv2.IMREAD_COLOR)
            cut_img_path = split_img_contours(img, path, filename)

print("Done")